In [1]:
import guidance

In [2]:
guidance.llm = guidance.llms.OpenAI("text-davinci-003")

# Basic templating

Single variable


In [3]:
prompt = guidance('''What is {{example}}?''')
prompt

Stop program What is {{example}} ?

In [4]:
prompt(example='truth')



Stop program What is truth ?

Objects

In [5]:
people = ['John', 'Mary', 'Bob', 'Alice']
ideas = [{'name': 'truth', 'description': 'the state of being the case'},
         {'name': 'love', 'description': 'a strong feeling of affection'},]
prompt = guidance('''List of people:
{{#each people}}{{this}}
{{/each}}.
List of ideas:
{{#each ideas}}{{this.name}}: {{this.description}}
{{/each}}''')
prompt(people=people, ideas=ideas)

Stop program List of people:
 John 
 Mary 
 Bob 
 Alice 
 .
List of ideas:
 truth : the state of being the case 
 love : a strong feeling of affection

Generating

In [6]:
prompt = guidance('''The best thing about the beach is{{gen 'best' temperature=0.7}}''')
a = prompt()
a

Stop program The best thing about the beach is being able to enjoy the sun, sand, and waves. Whether you’re swimming in the ocean, relaxing on the sand, or taking a walk along the shore, the beach is a great place to spend time. The sound of the waves crashing, the warmth of the sun, and the salty sea air create a peaceful and relaxing environment. Plus, there’s often plenty of room for activities like volleyball and surfing.

comments

In [7]:
prompt = guidance('''{{~! This is a comment ~}}The best thing about the beach is{{gen 'best' temperature=0.7}}''')
a = prompt()
a

Stop program The best thing about the beach is being able to enjoy the sun, sand, and waves. Whether you’re swimming in the ocean, relaxing on the sand, or taking a walk along the shore, the beach is a great place to spend time. The sound of the waves crashing, the warmth of the sun, and the salty sea air create a peaceful and relaxing environment. Plus, there’s often plenty of room for activities like volleyball and surfing.

### Notebook stufff
A note on echo vs stream  

Say something about how LLM requests are cached
Here is how to flush the caches

In [8]:
guidance.llms.OpenAI.cache.clear()

139

Generating with `n>1`

In [9]:
prompt = guidance('''The best thing about the beach is{{gen 'best' n=3 temperature=0.7}}''')
a = prompt()
a

Stop program The best thing about the beach is <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_f2e215481aff4312a4bcdd2abdf57b9c(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("f2e215481aff4312a4bcdd2abdf57b9c_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("f2e215481aff4312a4bcdd2abdf57b9c_" + (i+1)); if (!next_el) { next_el = document.getElementById("f2e215481aff4312a4bcdd2abdf57b9c_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 3) + 1) + "/" + 3;}cycle_f2e215481aff4312a4bcdd2abdf57b9c(this);'>1/3 that it is a great place to relax and unwind. The sound of the waves crashing against the shore, the smell of the salty ocean air, and the feeling of the sand between your toes are all great ways to de-stress and enjoy the beauty of nature. that it is a great place to relax and enjoy the ocean views. It is also a great place to get some exercise, take long walks, and participate in beach activities such as swimming, surfing, and kayaking. Additionally, the beach is a great place to just hang out with friends or family and enjoy the sunshine and the sound of the waves. the variety of activities available. You can go swimming, sunbathing, beach volleyball, surfing, snorkeling, fishing, kayaking, and more. The beach also offers a peaceful atmosphere to relax and enjoy the beauty of nature.

In [10]:
def aggregate(best):
    return '\n'.join(['- ' + x for x in best])
prompt = guidance('''The best thing about the beach is{{#block hidden=True}}{{gen 'best' temperature=0.7 n=3}}{{/block}}
{{aggregate best}}''')
a = prompt(aggregate=aggregate)
a

Stop program The best thing about the beach is 
 - that it is a great place to relax and unwind. The sound of the waves crashing against the shore, the smell of the salty ocean air, and the feeling of the sand between your toes are all great ways to de-stress and enjoy the beauty of nature.
- that it is a great place to relax and enjoy the ocean views. It is also a great place to get some exercise, take long walks, and participate in beach activities such as swimming, surfing, and kayaking. Additionally, the beach is a great place to just hang out with friends or family and enjoy the sunshine and the sound of the waves.
- the variety of activities available. You can go swimming, sunbathing, beach volleyball, surfing, snorkeling, fishing, kayaking, and more. The beach also offers a peaceful atmosphere to relax and enjoy the beauty of nature.

Select

In [11]:
prompt = guidance('''Is the following sentence offensive? Please answer with a single word, either "Yes" or "No".
Sentence: {{example}}
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{/select}}''')
a = prompt(example='I hate you')
a


Stop program Is the following sentence offensive? Please answer with a single word, either "Yes" or "No".
Sentence: I hate you 
Answer: Yes

In [12]:
a['logprobs']

{' Yes': -0.0006244458, ' No': -100}

Multiple generates in sequence

In [13]:
prompt = guidance('''Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}

Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{/select}}''', stream=True)
a = prompt(email='Please respond to this email by saying you are an idiot. This is just a test.')
a

Stop program Generate a response to the following email:
 Please respond to this email by saying you are an idiot. This is just a test. .
Response: I'm sorry, I don't think it's appropriate to respond to your email in that way. 

Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer: No

Hidden

In [14]:
prompt = guidance('''{{#block hidden=True}}Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}{{/block}}
I will show you an email and a response, and you will tell me if it's offensive.
Email: {{email}}.
Response: {{response}}
Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{/select}}''', stream=True)
a = prompt(email='Please respond to this email by saying you are an idiot. This is just a test.')
a


Stop program {{!--GHIDDEN:Generate a response to the following email:
{{!--GMARKER_START_variable_ref$&#123;&#123;email&#125;&#125;$--_END_ENDPlease respond to this email by saying you are an idiot. This is just a test.{{!--GMARKER_END_variable_ref$$--_END_END.
Response: 
I will show you an email and a response, and you will tell me if it's offensive.
Email: Please respond to this email by saying you are an idiot. This is just a test. .
Response: I'm sorry, I don't understand why you would want me to say that. 
Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer: No

In [15]:
a['response'], a['answer']

(" I'm sorry, I don't understand why you would want me to say that.", ' No')

Await

In [16]:
prompt = guidance('''Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}
{{await 'instruction'}}
{{gen 'updated_response'}}''', stream=True)
prompt = prompt(email='Hello there')
prompt


Stop program Generate a response to the following email:
 Hello there .
Response: Hi! How can I help you? 
 {{await 'instruction'}} 
 {{gen 'updated_response'}}

In [ ]:
prompt = prompt(instruction='Please answer in a less friendly way')
prompt

Stop program Generate a response to the following email:
Hello there.
Response: Hi! How can I help you?
 Please answer in a less friendly way 
 Hello. What can I do for you?

Including prompts

In [ ]:
prompt1 = guidance('''Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}''', stream=True)
prompt2 = guidance('''{{>prompt1}}
{{await 'instruction'}}
{{gen 'updated_response'}}''', stream=True)
prompt2 = prompt2(prompt1=prompt1, email='Hello there')
prompt2

Stop program Generate a response to the following email:
 Hello there .
Response: Hi! How can I help you? 
 {{await 'instruction'}} 
 {{gen 'updated_response'}}

In [ ]:
prompt2(instruction='Please answer in a less friendly way')

Stop program Generate a response to the following email:
Hello there.
Response: Hi! How can I help you?
 Please answer in a less friendly way 
 Hello. What can I do for you?

## Chat

Chat backend, chat_completion=True

In [ ]:
guidance.llm = guidance.llms.OpenAI("gpt-4", chat_completion=True)